In [2]:
import os
import json

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [72]:
def prepare_output(array):
    return 1/(1 + 10**(array/400))
    # return array/100

def prepare(file, train_part=0.9):
    df = pd.read_csv(file)
    data, evaluations = df.iloc[:,:-1], df.iloc[:,-1:]
    # print(data.shape)
    # print(data.to_numpy())

    # x_train = x_train.reshape(60000, 784).astype("float32") / 255
    # x_test = x_test.reshape(10000, 784).astype("float32") / 255

    # y_train = y_train.astype("float32")
    # y_test = y_test.astype("float32")

    # Reserve 10,000 samples for validation
    x_train = data[:int(len(data)*train_part)].to_numpy().reshape((-1, 12, 8, 8))
    y_train = prepare_output(evaluations[:int(len(data)*train_part)].to_numpy().flatten())
    x_test = data[int(len(data)*train_part):].to_numpy().reshape((-1, 12, 8, 8))
    y_test = prepare_output(evaluations[int(len(data)*train_part):].to_numpy().flatten())
    
    return x_train, y_train, x_test, y_test
# print(x_train.shape)
# print(x_test.shape)


In [73]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [74]:
class NeuralNetwork():
    def __init__(self, name):
        self.name = name
        self.optimizer = 'Adam'
        self.loss = 'mean_absolute_error'
        self.model = None
        self.results = ""
        # self.metrics= ['accuracy']

        # self.load_or_create()
        self.define()
        self.model.compile(optimizer=self.optimizer, loss=self.loss)
        
    def define(self):
        input_layer= tf.keras.Input(shape=(12, 8, 8))
        x = tf.keras.layers.Conv2D(filters=64,kernel_size = 2,strides = (2,2))(input_layer)
        x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,strides = (2,2))(x)
        x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,strides = (2,2))(x)
        x = tf.keras.layers.Flatten()(x)
        # x = tf.keras.layers.Dense(512,activation = 'relu')(input_layer)

        x = tf.keras.layers.Dense(512,activation = 'relu')(x)
        output = tf.keras.layers.Dense(1,activation = 'sigmoid')(x)

        model = tf.keras.Model(name=self.name, inputs=input_layer,outputs=output)
        self.model = model

    def load_or_create(self):
        path = f"models/{self.name}/architecutre.json"
        if not os.path.exists(path):
            self.define()
            return
        
        with open(f"{path}") as file:
            config = json.load(file)
            self.model = tf.keras.Model.from_config(config)
            
    
    def save_architecutre(self):
        path = f"models/{self.name}"
        if not os.path.exists(path):
            os.mkdir(path)
        
        with open(f"{path}/architecutre.json", 'w') as file:
            json.dump(self.model.get_config(), file, indent=4)
        
        with open(f"{path}/summary.txt", 'w') as file:
            def print_to_file(s):
                print()
            self.model.summary(print_fn=lambda x: print(x, file=file))
    
    def save_results(self):
        path = f"models/{self.name}"
        if not os.path.exists(path):
            os.mkdir(path)
        
        with open(f"{path}/results.txt", 'w') as file:
            file.write(self.results)


nn = NeuralNetwork(name="conv_3_512")
# nn.save_architecutre()


In [75]:
nn.model.summary()

Model: "conv_3_512"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 12, 8, 8)]        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 6, 4, 64)          2112      
                                                                 
 conv2d_10 (Conv2D)          (None, 3, 2, 128)         32896     
                                                                 
 conv2d_11 (Conv2D)          (None, 1, 1, 256)         131328    
                                                                 
 flatten_3 (Flatten)         (None, 256)               0         
                                                                 
 dense_24 (Dense)            (None, 512)               131584    
                                                                 
 dense_25 (Dense)            (None, 1)                 5

In [81]:
x_train, y_train, x_test, y_test = prepare('start_15_depth_10_moves_3.csv', 1)
print(x_train.shape, x_test.shape)
print(np.max(y_train))

history = nn.model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=25,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_split=0.3
)

(30663, 12, 8, 8) (0, 12, 8, 8)
1.0
Epoch 1/25
671/671 [==============================] - 4s 5ms/step - loss: 0.1539 - val_loss: 0.1759
Epoch 2/25
671/671 [==============================] - 3s 5ms/step - loss: 0.1517 - val_loss: 0.1803
Epoch 3/25
671/671 [==============================] - 3s 5ms/step - loss: 0.1516 - val_loss: 0.1655
Epoch 4/25
671/671 [==============================] - 3s 5ms/step - loss: 0.1510 - val_loss: 0.1791
Epoch 5/25
671/671 [==============================] - 3s 5ms/step - loss: 0.1502 - val_loss: 0.1635
Epoch 6/25
671/671 [==============================] - 3s 5ms/step - loss: 0.1502 - val_loss: 0.1824
Epoch 7/25
671/671 [==============================] - 3s 5ms/step - loss: 0.1481 - val_loss: 0.1713
Epoch 8/25
671/671 [==============================] - 3s 5ms/step - loss: 0.1481 - val_loss: 0.1734
Epoch 9/25
671/671 [==============================] - 3s 5ms/step - loss: 0.1487 - val_loss: 0.1856
Epoch 10/25
671/671 [==============================] - 3s 5ms/st

In [82]:
x_train, y_train, x_test, y_test = prepare('data1.csv', train_part=0)

print("Evaluate on test data")
results = nn.model.evaluate(x_test, y_test, batch_size=32)
print("test loss:", results)

Evaluate on test data
78/78 [==============================] - 0s 3ms/step - loss: 0.2726
test loss: 0.2726458013057709


In [29]:
x_train, y_train, x_test, y_test = prepare('data1.csv')

history = nn.model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=30,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_split = 0.2
)

Epoch 1/30
57/57 [==============================] - 1s 19ms/step - loss: 0.1382 - val_loss: 0.4597
Epoch 2/30
57/57 [==============================] - 1s 18ms/step - loss: 0.1010 - val_loss: 0.4734
Epoch 3/30
57/57 [==============================] - 1s 18ms/step - loss: 0.1008 - val_loss: 0.4751
Epoch 4/30
57/57 [==============================] - 1s 18ms/step - loss: 0.1000 - val_loss: 0.4714
Epoch 5/30
57/57 [==============================] - 1s 18ms/step - loss: 0.1001 - val_loss: 0.4773
Epoch 6/30
57/57 [==============================] - 1s 18ms/step - loss: 0.1007 - val_loss: 0.4653
Epoch 7/30
57/57 [==============================] - 1s 18ms/step - loss: 0.1000 - val_loss: 0.4814
Epoch 8/30
57/57 [==============================] - 1s 19ms/step - loss: 0.1000 - val_loss: 0.4767
Epoch 9/30
57/57 [==============================] - 1s 17ms/step - loss: 0.1000 - val_loss: 0.4795
Epoch 10/30
57/57 [==============================] - 1s 19ms/step - loss: 0.1002 - val_loss: 0.4689
Epoch 11/

KeyboardInterrupt: 

In [124]:
x_train, y_train, x_test, y_test = prepare('data2.csv')

history = nn.model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=100,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_split = 0.2
)

Epoch 1/100
24/24 [==============================] - 1s 31ms/step - loss: 0.0898 - val_loss: 0.0626
Epoch 2/100
24/24 [==============================] - 0s 13ms/step - loss: 0.0823 - val_loss: 0.0637
Epoch 3/100
24/24 [==============================] - 0s 12ms/step - loss: 0.0818 - val_loss: 0.0593
Epoch 4/100
24/24 [==============================] - 0s 14ms/step - loss: 0.0816 - val_loss: 0.0590
Epoch 5/100
24/24 [==============================] - 1s 27ms/step - loss: 0.0816 - val_loss: 0.0596
Epoch 6/100
24/24 [==============================] - 0s 9ms/step - loss: 0.0818 - val_loss: 0.0594
Epoch 7/100
24/24 [==============================] - 0s 10ms/step - loss: 0.0816 - val_loss: 0.0598
Epoch 8/100
24/24 [==============================] - 0s 10ms/step - loss: 0.0816 - val_loss: 0.0597
Epoch 9/100
24/24 [==============================] - 0s 12ms/step - loss: 0.0819 - val_loss: 0.0593
Epoch 10/100
24/24 [==============================] - 0s 11ms/step - loss: 0.0816 - val_loss: 0.0594


In [129]:
x_train, y_train, x_test, y_test = prepare('start_15_depth_10_moves_3.csv')

history = nn.model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=100,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_split = 0.2
)

Epoch 1/100
69/69 [==============================] - 2s 24ms/step - loss: 0.2487 - val_loss: 0.2325
Epoch 2/100
69/69 [==============================] - 1s 20ms/step - loss: 0.1555 - val_loss: 0.2203
Epoch 3/100
69/69 [==============================] - 1s 19ms/step - loss: 0.1554 - val_loss: 0.2254
Epoch 4/100
69/69 [==============================] - 1s 20ms/step - loss: 0.1557 - val_loss: 0.2231
Epoch 5/100
69/69 [==============================] - 1s 18ms/step - loss: 0.1552 - val_loss: 0.2150
Epoch 6/100
69/69 [==============================] - 1s 17ms/step - loss: 0.1560 - val_loss: 0.2178
Epoch 7/100
69/69 [==============================] - 1s 17ms/step - loss: 0.1554 - val_loss: 0.2275
Epoch 8/100
69/69 [==============================] - 1s 19ms/step - loss: 0.1561 - val_loss: 0.2204
Epoch 9/100
69/69 [==============================] - 1s 18ms/step - loss: 0.1559 - val_loss: 0.2251
Epoch 10/100
69/69 [==============================] - 1s 15ms/step - loss: 0.1552 - val_loss: 0.2120

In [128]:
print("Evaluate on test data")
results = nn.model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = nn.model.predict(x_test[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
2/2 [==============================] - 0s 12ms/step - loss: 0.0765
test loss, test acc: 0.07646416872739792
Generate predictions for 3 samples
1/1 [==============================] - 0s 37ms/step
predictions shape: (3, 12, 8, 1)
